For this code ai helped me a lot, especially for the merging between datasets

In [5]:
import pandas as pd

In [6]:
df = pd.read_excel(r'full_normalized_dataset_correct.xlsx')

In [7]:
df.columns

Index(['speechID', 'memberID', 'partyID', 'constID', 'title', 'date',
       'member_name', 'party_name', 'const_name', 'speech', 'ni_score',
       'five_year_period', 'ni_similarity', 'clean_speech', 'embeddings',
       'Anglo-Irish Agreement', 'Governance UK/Ireland', 'topic_4_prob',
       'Cross-Border Legal Frameworks', 'topic_6_prob', 'topic_8_prob',
       'topic_9_prob', 'North-South Coordination',
       'Political Dialogue & Prisoner Releases', 'topic_12_prob',
       'topic_13_prob', 'Good Friday Agreement Protections', 'topic_16_prob',
       'topic_17_prob', 'main_topic', 'nationalist_pop', 'unionist_pop',
       'violence_deaths', 'Paramilitary Activities & Ceasefires',
       'Agreement Negotiation & Implementation',
       'Security Reforms (Decommissioning & Policing)',
       'Human Rights & Prisoner Policies',
       'Terrorism, Paramilitary Violence, and Security Cooperation',
       'Peace Agreements and Political Settlements',
       'UK/Ireland & Cross-Border R

Adding border_consituency variable

In [8]:
df["const_name"].unique()

array(['Athlone-Longford', 'Waterford', 'Dublin County', 'Kerry North',
       'Dublin South-West', 'Monaghan', 'Monaghan South', 'Wexford South',
       'Donegal East', 'Dublin South', 'Clare East', 'Louth',
       'Dublin (Clontarf)', 'Limerick', 'Donegal West', 'Cork West',
       'Meath-Westmeath', 'Dublin North-West', 'Cork South-East',
       'Dublin Townships', 'Cork South', 'Tipperary', 'Kerry South',
       'Mayo North', 'Limerick East', 'Wexford',
       'Dún Laoghaire and Rathdown', 'Cork North', 'Tipperary South',
       'Dublin North-East', 'Limerick West', 'Donegal North-East',
       'Sligo-Leitrim', 'Cork Borough', 'Longford -Westmeath',
       'Dublin South-Central', 'Cavan', 'Clare', 'Dublin South-East',
       'Tipperary North', 'Leix-Offaly', 'Cork Mid', 'Mayo East',
       'Cork City North-West', 'Cork City South-East', 'Cork East',
       'Laoighis-Offaly', 'Dublin County South', 'Galway West', 'Meath',
       'Cork North-East', 'Dublin County North', 'Kildare', '

In [9]:
border_constituencies = {
    'Donegal', 'Donegal East', 'Donegal West', 'Donegal North-East', 'Donegal-Leitrim',
    'Donegal South-West', 'Cavan', 'Monaghan', 'Monaghan South', 'Cavan-Monaghan',
    'Leitrim', 'Sligo-Leitrim', 'Louth'
}

df['border_constituency'] = df['const_name'].apply(lambda x: 1 if x in border_constituencies else 0)


Merging with Comparative Manifesto Project for RILE scores of parties

In [10]:
df['party_name'].unique()

array(['Fianna Fáil', 'Fine Gael', 'Sinn Féin'], dtype=object)

In [11]:
mapping_code = {"Sinn Féin": 53951, "Fine Gael": 53520, "Fianna Fáil": 53620}

df['partycode'] = df['party_name'].map(mapping_code)
df.columns

Index(['speechID', 'memberID', 'partyID', 'constID', 'title', 'date',
       'member_name', 'party_name', 'const_name', 'speech', 'ni_score',
       'five_year_period', 'ni_similarity', 'clean_speech', 'embeddings',
       'Anglo-Irish Agreement', 'Governance UK/Ireland', 'topic_4_prob',
       'Cross-Border Legal Frameworks', 'topic_6_prob', 'topic_8_prob',
       'topic_9_prob', 'North-South Coordination',
       'Political Dialogue & Prisoner Releases', 'topic_12_prob',
       'topic_13_prob', 'Good Friday Agreement Protections', 'topic_16_prob',
       'topic_17_prob', 'main_topic', 'nationalist_pop', 'unionist_pop',
       'violence_deaths', 'Paramilitary Activities & Ceasefires',
       'Agreement Negotiation & Implementation',
       'Security Reforms (Decommissioning & Policing)',
       'Human Rights & Prisoner Policies',
       'Terrorism, Paramilitary Violence, and Security Cooperation',
       'Peace Agreements and Political Settlements',
       'UK/Ireland & Cross-Border R

In [12]:
cmp_df = pd.read_csv(r'MPDataset_MPDS2024a.csv')

C:\Users\mathi\AppData\Local\Temp\ipykernel_29348\4001346104.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  cmp_df = pd.read_csv(r'MPDataset_MPDS2024a.csv')


In [13]:
cmp_df.columns

Index(['country', 'countryname', 'oecdmember', 'eumember', 'edate', 'date',
       'party', 'partyname', 'partyabbrev', 'parfam',
       ...
       'per608_3', 'per703_1', 'per703_2', 'rile', 'planeco', 'markeco',
       'welfare', 'intpeace', 'datasetversion', 'id_perm'],
      dtype='object', length=175)

In [ ]:
# Step 1: Prepare the CMP dataset
cmp_df = cmp_df[["country", "party", "partyname", "edate", "rile"]]
cmp_df = cmp_df[(cmp_df["country"] == 53) & (cmp_df["party"].isin([53951, 53520, 53620]))]

# Extract the year from election dates
cmp_df['year'] = pd.to_datetime(cmp_df['edate'], dayfirst=True).dt.year
cmp_df.rename(columns={'party': 'partycode'}, inplace=True)

# Sort to get the latest rile available for that year
cmp_df = cmp_df.sort_values(['partycode', 'year', 'edate'])
cmp_df = cmp_df.drop_duplicates(subset=['partycode', 'year'], keep='last')

# Step 2: Build the year-party grid
complete_years = pd.DataFrame({
    'year': range(cmp_df['year'].min(), cmp_df['year'].max() + 1),
    'key': 1
})
parties = pd.DataFrame({
    'partycode': cmp_df['partycode'].unique(),
    'key': 1
})
grid = complete_years.merge(parties, on='key').drop(columns='key')

# Step 3: Merge and forward-fill missing values
cmp_df_completo = grid.merge(
    cmp_df[['partycode', 'year', 'rile']],
    on=['partycode', 'year'],
    how='left'
).sort_values(['partycode', 'year']).ffill()


In [ ]:
# Find and print the common column names between the two dataframes
common_columns = set(df.columns).intersection(set(cmp_df.columns))
print(common_columns)

{'partycode', 'year'}


In [ ]:
# Ensure that the 'partycode' and 'year' columns in both dataframes are of integer type
df['partycode'] = df['partycode'].astype(int)
cmp_df_completo['partycode'] = cmp_df_completo['partycode'].astype(int)
df['year'] = df['year'].astype(int)
cmp_df_completo['year'] = cmp_df_completo['year'].astype(int)


In [ ]:
# Merge the main dataframe (df) with the CMP dataset (cmp_df_completo)
# using 'partycode' and 'year' as keys, performing a left join.
df_controls = df.merge(
    cmp_df_completo,
    on=['partycode', 'year'],
    how='left'
)

In [18]:
print(f"Righe originali: {len(df)}")
print(f"Righe dopo merge: {len(df_controls)}")
print("Valori mancanti in rile:", df_controls['rile'].isna().sum())


Righe originali: 4619
Righe dopo merge: 4619
Valori mancanti in rile: 0


In [19]:
# 1. Rimuovi i topic già uniti nei cluster
topic_prob_cols_to_remove = [
    'topic_4_prob', 'topic_6_prob', 'topic_8_prob',
    'topic_9_prob', 'topic_12_prob', 'topic_13_prob',
    'topic_16_prob', 'topic_17_prob'
]

df_cleaned = df_controls.drop(columns=topic_prob_cols_to_remove)

# 2. Riordina le colonne
ordered_columns = [
    # Info discorso
    'speechID', 'memberID', 'partyID', 'constID', 'title', 'date', 'year',
    'member_name', 'party_name', 'const_name', 'speech', 'clean_speech',

    # Variabili di contenuto
    'main_topic',
    'Anglo-Irish Agreement', 'Governance UK/Ireland',
    'Cross-Border Legal Frameworks', 'North-South Coordination',
    'Political Dialogue & Prisoner Releases',
    'Good Friday Agreement Protections',
    'Paramilitary Activities & Ceasefires',
    'Agreement Negotiation & Implementation',
    'Security Reforms (Decommissioning & Policing)',
    'Human Rights & Prisoner Policies',
    'Terrorism, Paramilitary Violence, and Security Cooperation',
    'Peace Agreements and Political Settlements',
    'UK/Ireland & Cross-Border Relations',

    # Variabili di contesto
    'ni_score', 'ni_similarity', 'violence_deaths',
    'nationalist_pop', 'unionist_pop',

    # Controlli
    'five_year_period', 'border_constituency', 'partycode', 'rile',

    # Extra (se vuoi tenerlo per NLP)
    'embeddings'
]

# Applica il nuovo ordine
df_cleaned = df_cleaned[ordered_columns]


In [20]:
df_cleaned.columns

Index(['speechID', 'memberID', 'partyID', 'constID', 'title', 'date', 'year',
       'member_name', 'party_name', 'const_name', 'speech', 'clean_speech',
       'main_topic', 'Anglo-Irish Agreement', 'Governance UK/Ireland',
       'Cross-Border Legal Frameworks', 'North-South Coordination',
       'Political Dialogue & Prisoner Releases',
       'Good Friday Agreement Protections',
       'Paramilitary Activities & Ceasefires',
       'Agreement Negotiation & Implementation',
       'Security Reforms (Decommissioning & Policing)',
       'Human Rights & Prisoner Policies',
       'Terrorism, Paramilitary Violence, and Security Cooperation',
       'Peace Agreements and Political Settlements',
       'UK/Ireland & Cross-Border Relations', 'ni_score', 'ni_similarity',
       'violence_deaths', 'nationalist_pop', 'unionist_pop',
       'five_year_period', 'border_constituency', 'partycode', 'rile',
       'embeddings'],
      dtype='object')

In [21]:
df_cleaned.to_excel('normalized_controls_dataset_correct.xlsx', index=False)